In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import dmc
import dmc.genetic as gen
import dmc.features
from dmc.classifiers import Forest, NaiveBayes, SVM, TreeBag
import pandas as pd
import numpy as np



In [5]:
df = pd.read_csv('data/processed.csv', sep = ',')
dummy = pd.DataFrame(columns = df.columns.tolist())
df, dummy = dmc.features.add_dependent_features(df, dummy)

In [28]:
classifierSet = {'Forest' : Forest, 'NaiveBayes' : NaiveBayes, 'SVM' : SVM}

notUsed = ['returnQuantity', 'orderID', 'orderDate', 'customerID', 'Unnamed: 0', 'Unnamed', 'ID']
uk = {}
uk['Article'] = ['articleID'] + notUsed
uk['Customer'] = ['customerReturnProb', 'customerID'] + notUsed
uk['Voucher'] = ['voucherID '] + notUsed

uk['ArticleProductgroup'] = ['productGroup', 'productGroupReturnProb'] + uk['Article']
uk['ArticleCustomer'] = uk['Article'] + uk['Customer']
uk['ArticleVoucher'] = uk['Article'] + uk['Voucher']
uk['CustomerVoucher'] =  uk['Customer'] + uk['Voucher']

uk['ArticleProductgroupCustomer'] = uk['ArticleProductgroup'] + uk['Customer']
uk['ArticleProductgroupVoucher'] =  uk['ArticleProductgroup'] + uk['Voucher']
uk['ArticleCustomerVoucher'] = uk['Article'] + uk['Customer'] + uk['Voucher']

uk['ArticleProductgroupCustomerVoucher'] = uk['Customer'] + uk['Voucher'] + uk['ArticleProductgroup']

uk['None'] = notUsed


In [ ]:
for identifier, missedFts in uk.items():
    featureset = gen.prepareFeatureList(df, missedFts)
    for classifierName, classifier in classifierSet.items():
        sets, score = gen.playGod(classifierName, identifier, classifier, df, featureset)
        sets.to_csv('gen_algo_sets/Sets_' + classifierName + '_' + identifier + '.csv', sep=',')
        score.to_csv('gen_algo_score/Sets_' + classifierName + '_' + identifier + '.csv', sep=',')

genes: 20 generations: 15 keep: 4 mutationProb: 0.1
